## Import modules, classes, and functions

In [12]:
import numpy as np
from scipy import stats
from dataclasses import dataclass
from dataclasses import field

from LogisticMapLCE import logistic_lce
from HenonMapLCE import henon_lce
from IkedaMapLCE import ikeda_lce
from TinkerbellMapLCE import tinkerbell_lce

from TimeSeriesHVG import TimeSeriesHVG as TSHVG
from TimeSeriesMergeTree import TimeSeriesMergeTree as TSMT
from TimeSeriesPersistence import TimeSeriesPersistence as TSPH

## Configure the experiments

In [2]:
@dataclass
class BifurcationConfig:
    """Configure a chaotic map over a range of its control parameter values."""

    map_name: str = field(metadata={"description": "name of the chaotic map"})
    map_generator: callable = field(
        metadata={
            "description": "function to generate chaotic trajectories and lyapunov estimates"
        }
    )
    control_param_name: str = field(
        metadata={"description": "name of the variable control parameter"}
    )
    control_param_min: float = field(
        metadata={"description": "minimum value of control parameter"}
    )
    control_param_max: float = field(
        metadata={"description": "maximum value of control parameter"}
    )
    control_param_num: int = field(
        metadata={"description": "number of control parameters to sample in range"}
    )
    initial_state: dict = field(
        metadata={"description": "names and values of the initial state variables"}
    )
    fixed_params: dict = field(
        default_factory=lambda: dict(),
        metadata={"description": "names and values of fixed control parameters"},
    )
    SEED: int = field(
        default=42,
        metadata={"description": "random seed for control parameter generation"},
        kw_only=True,
    )
    projection_for_time_series: int = field(
        default=0,
        metadata={"description": "which of the state dimensions to use for time series"},
    )

    @property
    def control_param_values(self):
        from numpy.random import MT19937
        from numpy.random import RandomState
        from numpy.random import SeedSequence

        SEED = self.SEED
        randomState = RandomState(MT19937(SeedSequence(SEED)))
        param_values = np.sort(
            randomState.uniform(
                self.control_param_min, self.control_param_max, self.control_param_num
            )
        )
        return param_values

    @property
    def bifurcation_params(self):
        return [self.fixed_params | {self.control_param_name: v} for v in self.control_param_values]

In [3]:
@dataclass
class TrajectoryLceConfig:
    system: BifurcationConfig = field(
        metadata={
            "description": "The bifurcation configuration to use to generate data"
        }
    )
    nTransients: int = field(
        default=100,
        metadata={
            "description": "map iterates to ignore to ensure trajectories are on the attractor"
        },
        kw_only=True,
    )
    nIterates: int = field(
        default=1000,
        metadata={"description": "length of trajectory on attractor to generate"},
        kw_only=True,
    )
    nTransients_lce: int = field(
        default=200,
        metadata={
            "description": "map iterates to ignore when estimating lyapunov exponents"
        },
        kw_only=True,
    )
    nIterates_lce: int = field(
        default=10000,
        metadata={
            "description": "length of trajectory to use when estimating lyapunov exponents"
        },
        kw_only=True,
    )
    includeTrajectory: bool = field(
        default=True,
        metadata={
            "description": "return a trajectory as well as the lyapunov exponents"
        },
        kw_only=True,
    )
    fullLceSpectrum: bool = field(
        default=False,
        metadata={"description": "include the non-maximal Lyapunov exponents as well"},
        kw_only=True,
    )


In [4]:
class BifurcationExperiment:
    def __init__(self, config: TrajectoryLceConfig) -> None:
        self.config = config
        self.system = self.config.system
        self._data = []

    def generate_data(self):
        self._data = []
        map_generator = self.system.map_generator
        bifurcation_params = self.system.bifurcation_params
        initial_state = self.system.initial_state
        map_kwparams = dict(
            nTransients=self.config.nTransients,
            nIterates=self.config.nIterates,
            nTransients_lce=self.config.nTransients_lce,
            nIterates_lce=self.config.nIterates_lce,
            includeTrajectory=self.config.includeTrajectory,
            fullLceSpectrum=self.config.fullLceSpectrum,
        )
        for params in bifurcation_params:
            self._data.append(map_generator(mapParams=params, initialState=initial_state, **map_kwparams))

    @property
    def data(self):
        if len(self._data) == 0:
            self.generate_data()
        return self._data
    
    @property
    def lces(self):
        lce = lambda experimental_data: experimental_data["lce"][0]
        return np.array([lce(data) for data in self._data])

    @property
    def time_series(self):
        d = self.system.projection_for_time_series
        return [data["trajectory"][:,d] for data in self._data]



In [13]:
from typing import Literal

@dataclass
class RepresentationConfig:
    name: Literal["hvg", "mt", "ph"]

    constructors = {
        "hvg": TSHVG,
        "mt": TSMT,
        "ph": TSPH,
    }
    
    @property
    def constructor(self):
        return self.constructors[self.name]
    
    kwargs: dict[str, any]

    

In [5]:
NUM_TRAJECTORIES_PER_SYSTEM = 10

logistic_config = BifurcationConfig(
    map_name="logistic",
    map_generator=logistic_lce,
    control_param_name="r",
    control_param_min=3.5,
    control_param_max=4.0,
    control_param_num=NUM_TRAJECTORIES_PER_SYSTEM,
    initial_state=dict(x=0.2)
)

henon_config = BifurcationConfig(
    map_name="henon",
    map_generator=henon_lce,
    control_param_name="a",
    control_param_min=0.8,
    control_param_max=1.4,
    control_param_num=NUM_TRAJECTORIES_PER_SYSTEM,
    fixed_params=dict(b=0.3),
    initial_state=dict(x=0.1, y=0.3)
)


In [ ]:
hvg_config = RepresentationConfig(
    name="hvg",
    kwargs=dict(
        DEGREE_DISTRIBUTION_MAX_DEGREE=100,
        DEGREE_DISTRIBUTION_DIVERGENCE_P_VALUE=1.0,
        directed=None,
        weighted=None,
        penetrable_limit=0,
    )
)

mt_config = RepresentationConfig(
    name="mt",
    kwargs=dict(
        INTERLEAVING_DIVERGENCE_MESH=0.5,
        DMT_ALPHA=0.5,
        DISTRIBUTION_VECTOR_LENGTH=100,
    )
)

ph_config = RepresentationConfig(
    name="ph",
    kwargs=dict(
        ENTROPY_SUMMARY_RESOLUTION=100,
        BETTI_CURVE_RESOLUTION=100,
        BETTI_CURVE_NORM_P_VALUE=1.0,
        SILHOUETTE_RESOLUTION=100,
        SILHOUETTE_WEIGHT=1,
        LIFESPAN_CURVE_RESOLUTION=100,
        IMAGE_BANDWIDTH=0.2,
        IMAGE_RESOLUTION=20,
        ENTROPY_SUMMARY_DIVERGENCE_P_VALUE=2.0,
        PERSISTENCE_STATISTICS_DIVERGENCE_P_VALUE=2.0,
        WASSERSTEIN_DIVERGENCE_P_VALUE=1.0,
        BETTI_CURVE_DIVERGENCE_P_VALUE=1.0,
        PERSISTENCE_SILHOUETTE_DIVERGENCE_P_VALUE=2.0,
        PERSISTENCE_LIFESPAN_DIVERGENCE_P_VALUE=2.0,
    )
)

In [6]:

system_configs: list[BifurcationConfig] = [logistic_config, henon_config]
experiments: dict[str, BifurcationExperiment] = dict()

for system_config in system_configs:
    experiment_config = TrajectoryLceConfig(system=system_config)
    bifurcation_experiment = BifurcationExperiment(config=experiment_config)
    experiments[bifurcation_experiment.system.map_name] = bifurcation_experiment

In [7]:
for experiment in experiments.values():
    experiment.generate_data()

In [8]:
experiments["logistic"].data

[{'system': 'logistic',
  'params': {'r': 3.5286848908533344},
  'initial': {'x': 0.2},
  'iterates': {'trajectory': {'nTransients': 100, 'nIterates': 1000},
   'lce': {'nTransients': 200, 'nIterates': 10000}},
  'lce': (-0.10634193121270565,),
  'trajectory': array([[0.82194423],
         [0.51642977],
         [0.8812187 ],
         ...,
         [0.88121862],
         [0.36935578],
         [0.82194394]])},
 {'system': 'logistic',
  'params': {'r': 3.7403820276201603},
  'initial': {'x': 0.2},
  'iterates': {'trajectory': {'nTransients': 100, 'nIterates': 1000},
   'lce': {'nTransients': 200, 'nIterates': 10000}},
  'lce': (-0.0631729477843723,),
  'trajectory': array([[0.49512363],
         [0.93500656],
         [0.22730036],
         ...,
         [0.65694116],
         [0.84296792],
         [0.49512561]])},
 {'system': 'logistic',
  'params': {'r': 3.7709969465031374},
  'initial': {'x': 0.2},
  'iterates': {'trajectory': {'nTransients': 100, 'nIterates': 1000},
   'lce': {'nTr

In [9]:
experiments["logistic"].lces

array([-0.10634193, -0.06317295,  0.39735755,  0.42847687,  0.43455378,
       -0.13680436, -0.01506717,  0.43273952, -0.07622925,  0.55053153])

In [10]:
experiments["logistic"].tshvgs

In [11]:
experiments["henon"].tshvgs